# 🐋 Kuramoto Model Simulation for Whale Vocal Synchrony (G1, G2, G3)
This notebook simulates phase synchrony among whales in each group using the Kuramoto model. We treat each whale as an oscillator, estimate natural frequencies from call intervals, and model coupling via vocal responses.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
%matplotlib inline

In [ ]:
# Load the vocalization dataset and normalize call types
df = pd.read_csv("PrepData.csv")
df["CallType"] = df["CallType"].fillna("InitiationCall")

In [ ]:
# Define a function to simulate Kuramoto dynamics for a given group
def simulate_kuramoto(group_label):
    df_group = df[df["Group"] == group_label].dropna(subset=["WhaleID", "StartTime"])
    whales = df_group["WhaleID"].unique()
    n_whales = len(whales)
    whale_to_idx = {whale: i for i, whale in enumerate(whales)}

    # Estimate natural frequencies
    call_intervals = []
    for whale, group in df_group.groupby("WhaleID"):
        times = group.sort_values("StartTime")["StartTime"].values
        if len(times) > 1:
            intervals = np.diff(times)
            call_intervals.append((whale, 1 / np.mean(intervals)))
        else:
            call_intervals.append((whale, 1.0))
    omega = np.array([dict(call_intervals).get(whale, 1.0) for whale in whales])

    # Build coupling matrix A[i, j] = j influences i
    A = np.zeros((n_whales, n_whales))
    for bout_id, bout_df in df_group.groupby("bout"):
        bout_sorted = bout_df.sort_values("StartTime")
        initiator_row = bout_sorted[bout_sorted["CallType"] == "InitiationCall"]
        responders = bout_sorted[bout_sorted["CallType"] == "ResponseCall"]
        if not initiator_row.empty:
            initiator = initiator_row.iloc[0]["WhaleID"]
            for _, responder_row in responders.iterrows():
                responder = responder_row["WhaleID"]
                if initiator != responder:
                    i = whale_to_idx[responder]
                    j = whale_to_idx[initiator]
                    A[i, j] += 1
    if A.max() > 0:
        A /= A.max()

    # Simulate Kuramoto dynamics
    T = 100
    dt = 0.1
    steps = int(T / dt)
    K = 2.0
    theta = np.zeros((steps, n_whales))
    theta[0] = np.random.uniform(0, 2 * np.pi, n_whales)

    for t in range(1, steps):
        dtheta = omega + (K / n_whales) * np.sum(A * np.sin(theta[t-1] - theta[t-1][:, None]), axis=1)
        theta[t] = theta[t-1] + dt * dtheta

    r = np.abs(np.mean(np.exp(1j * theta), axis=1))
    time = np.linspace(0, T, steps)

    return {
        "group": group_label,
        "whales": whales.tolist(),
        "omega": omega.tolist(),
        "A": A.tolist(),
        "theta": theta.tolist(),
        "r": r.tolist(),
        "time": time.tolist()
    }

In [ ]:
# Run the simulation for G1, G2, and G3
results = {}
for group_label in ["G1", "G2", "G3"]:
    results[group_label] = simulate_kuramoto(group_label)

# Save each group's result to a JSON file
for group_label, data in results.items():
    with open(f"kuramoto_simulation_{group_label}.json", "w") as f:
        json.dump(data, f)

list(results.keys())